In [1]:
!pip install numpy
!pip install ultralytics
!pip install torch torchvision torchaudio

In [1]:
import numpy as np 
import os
from xml.dom.minidom import parse
from ultralytics import YOLO
import torch

In [2]:
cls_to_idx={'RBC':0,'WBC':1,'Platelets':2}

In [3]:
with open('detect_count_yolov8/BCCD/ImageSets/Main/train.txt','r') as file:
    TrainLines1=file.readlines()

with open('detect_count_yolov8/BCCD/ImageSets/Main/trainval.txt','r') as file:
    TrainLines2=file.readlines()

TrainLines=TrainLines1+TrainLines2
TrainLines=[line.strip() for line in TrainLines]

In [4]:
def cord_converter(size,img_box):
    x1=int(img_box[0])
    y1=int(img_box[1])
    x2=int(img_box[2])
    y2=int(img_box[3])
    dw=np.float32(1/int(size[0]))
    dh=np.float32(1/int(size[1]))
    w=x2-x1
    h=y2-y1 
    x=x1+(w/2)
    y=y1+(h/2)
    x=x*dw
    y=y*dh
    w=w*dw
    h=h*dw
    return [x,y,w,h]

In [5]:
def save_file(picName,size, img_box):
    path='detect_count_yolov8/BCCD/labels/train/'+picName+'.txt'
    file=open(path,'a+')

    for box in img_box:
        cls_num=cls_to_idx[box[0]]
        new_box=cord_converter(size, box[1:])
        file.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file.flush()
    file.close()

In [6]:
def get_xml_data(file):
    dom=parse(file)
    root=dom.documentElement

    img_name=root.getElementsByTagName('filename')[0].childNodes[0].data
    img_size=root.getElementsByTagName('size')[0]

    img_w=img_size.getElementsByTagName('width')[0].childNodes[0].data
    img_h=img_size.getElementsByTagName('height')[0].childNodes[0].data
    img_c=img_size.getElementsByTagName('depth')[0].childNodes[0].data

    objects=root.getElementsByTagName('object')
    img_box=[]

    for box in objects:
        cls_name=box.getElementsByTagName('name')[0].childNodes[0].data
        x1=box.getElementsByTagName('xmin')[0].childNodes[0].data
        y1=box.getElementsByTagName('ymin')[0].childNodes[0].data
        x2=box.getElementsByTagName('xmax')[0].childNodes[0].data
        y2=box.getElementsByTagName('ymax')[0].childNodes[0].data
        img_box.append([cls_name,x1,y1,x2,y2])

    save_file(img_name.strip('.jpg'),[img_w,img_h],img_box)

In [7]:
files=os.listdir('detect_count_yolov8/BCCD/Annotations/')
for file in files:
    if file.strip('.xml') in TrainLines:
        file=os.path.join('detect_count_yolov8/BCCD/Annotations/',file)
        get_xml_data(file)

In [ ]:
model = YOLO("detect_count_yolov8/yolov8n.pt")
results = model.train(
    data="detect_count_yolov8/BCCD/config.yaml",
    epochs=100,
    device="mps"
)

New https://pypi.org/project/ultralytics/8.4.6 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.3 🚀 Python-3.11.14 torch-2.9.1 MPS (Apple M2 Max)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=detect_count_yolov8/BCCD/config.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=detect_count_yolov8/yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train7, nbs=64, nm

In [11]:
!yolo task=detect mode=predict model=runs\detect\train2\weights\best.pt conf=0.25 source=detect_count_yolov8\BCCD\images\test

Traceback (most recent call last):
  File "/opt/anaconda3/envs/traffic-tf/bin/yolo", line 7, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/opt/anaconda3/envs/traffic-tf/lib/python3.11/site-packages/ultralytics/cfg/__init__.py", line 955, in entrypoint
    model = YOLO(model, task=task)
            ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/traffic-tf/lib/python3.11/site-packages/ultralytics/models/yolo/model.py", line 76, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "/opt/anaconda3/envs/traffic-tf/lib/python3.11/site-packages/ultralytics/engine/model.py", line 144, in __init__
    self._load(model, task=task)
  File "/opt/anaconda3/envs/traffic-tf/lib/python3.11/site-packages/ultralytics/engine/model.py", line 283, in _load
    self.model, self.ckpt = load_checkpoint(weights)
                            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/traffic-tf/lib/python3.11/site-packages/ultralytics/nn/tasks